In [ ]:
!pip install flaml

In [ ]:
import pandas as pd
import numpy as np
import datatable as dt
from flaml import AutoML

In [ ]:
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [ ]:
%%time
train = dt.fread('../input/tabular-playground-series-nov-2021/train.csv').to_pandas().drop('id', axis=1)
train = reduce_memory_usage(train)
test = dt.fread('../input/tabular-playground-series-nov-2021/test.csv').to_pandas().drop('id', axis=1)
test = reduce_memory_usage(test)
ss = dt.fread('../input/tabular-playground-series-nov-2021/sample_submission.csv').to_pandas()
ss = reduce_memory_usage(ss)

In [ ]:
bool_cols_train = []
for i, col in enumerate(train.columns):
    if train[col].dtypes == bool:
        bool_cols_train.append(i)
    
print(bool_cols_train)

In [ ]:
bool_cols_test = []
for i, col in enumerate(test.columns):
    if train[col].dtypes == bool:
        bool_cols_test.append(i)
    
print(bool_cols_test)

In [ ]:
train.iloc[:, bool_cols_train] = train.iloc[:, bool_cols_train].astype(int)
test.iloc[:, bool_cols_test] = test.iloc[:, bool_cols_test].astype(int)

In [ ]:
print("Train set shape", train.shape, "\n", "Test set shape", test.shape)

In [ ]:
X = train.drop('target', axis=1).values
y = train['target'].values
X_test = test.values

In [ ]:
del train

# FLAML

In [ ]:
"""Initializing AutoMl instance
"""
automl = AutoML()

"""Specify automl goal and constraint
"""
automl_settings = {
    "time_budget" : 3600*6, # in seconds
    "metric" : 'roc_auc',
    "task" : 'classification',
    "log_file_name" : "tabular.log"
}

automl.fit(X, y, **automl_settings)
preds = automl.predict_proba(X_test)

ss['target'] = preds[:, -1]
ss.to_csv('./flaml.csv', index=False)
ss.head()